In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [21]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
#df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)


In [22]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################


df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
#df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
#df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 84 columns):
cant_conversions                                38829 non-null float64
cant_checkouts                                  38829 non-null float64
cant_viewed_product                             38829 non-null float64
cant_searched_product                           38829 non-null float64
cant_ad_campaign_hit                            38829 non-null float64
ad_campaign_hit                                 38829 non-null bool
cant_lead                                       38829 non-null float64
lead                                            38829 non-null bool
cant_brand_listing                              38829 non-null float64
cant_visited_site                               38829 non-null float64
cant_generic_listing                            38829 non-null float64
total_sesiones                                  38829 non-null int64
promedio_eventos_por_sesion               

In [23]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 85 columns):
cant_conversions                                19414 non-null float64
cant_checkouts                                  19414 non-null float64
cant_viewed_product                             19414 non-null float64
cant_searched_product                           19414 non-null float64
cant_ad_campaign_hit                            19414 non-null float64
ad_campaign_hit                                 19414 non-null bool
cant_lead                                       19414 non-null float64
lead                                            19414 non-null bool
cant_brand_listing                              19414 non-null float64
cant_visited_site                               19414 non-null float64
cant_generic_listing                            19414 non-null float64
total_sesiones                                  19414 non-null int64
promedio_eventos_por_sesion               

In [24]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['cant_conversions', 'cant_checkouts', 'cant_viewed_product', 'cant_searched_product', 'cant_ad_campaign_hit', 'ad_campaign_hit', 'cant_lead', 'lead', 'cant_brand_listing', 'cant_visited_site', 'cant_generic_listing', 'total_sesiones', 'promedio_eventos_por_sesion', 'timestamp', 'min_timestamp', 'sum_timestamp', 'prom_timestamp', 'ult_view', 'mean_sku', 'min_sku', 'suma_sku', 'compro_apple', 'compro_samsung', 'compro_motorola', 'conte_10', 'vio_mas_apple', 'vio_mas_samsung', 'vio_mas_motorola', 'check_apple', 'check_samsung', 'check_motorola', 'city_Sao Paulo', 'city_S\xc3\xa3o Paulo', 'staticpage_AboutUs', 'staticpage_Conditions', 'staticpage_CustomerService', 'staticpage_FaqEcommerce', 'staticpage_PrivacyEcommerce', 'staticpage_Quiosks', 'staticpage_TermsAndConditionsEcommerce', 'staticpage_TermsAndConditionsReturnEcommerce', 'staticpage_black_friday', 'staticpage_club-trocafone', 'staticpage_galaxy-s8', 'staticpage_how-to-buy', 'staticpage_how-to-sell', 'staticpage_trust-trocafone',

In [25]:
labels['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

In [26]:
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, min_samples_split=150,\
                            class_weight='balanced')

#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [27]:
rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

0.80916816894154009

In [28]:
Y_pred = rf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.787147595357


In [29]:
#  Esto nos tira data sobre la precisión, cuántas de las cosas que clasificamos salieron bien clasificadas, y f1

Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.98      0.81      0.89      3685
          1       0.18      0.76      0.29       198

avg / total       0.94      0.81      0.86      3883



In [30]:
print(confusion_matrix(Y_truths, Y_pred))

[[2991  694]
 [  47  151]]


In [31]:
feature_importances = pd.DataFrame(rf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [32]:
feature_importances

,importance
cant_checkouts,2.123270e-01
prom_timestamp,7.561854e-02
timestamp,6.657356e-02
min_timestamp,6.204450e-02
ult_view,5.879372e-02
sum_timestamp,4.591168e-02
event_checkout,4.234664e-02
cant_viewed_product,3.756320e-02
suma_sku,3.722911e-02
check_apple,3.552255e-02


## Summit

In [33]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [34]:
df_events

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,lead,cant_brand_listing,cant_visited_site,...,sku_8443.0,sku_723.0,sku_6023.0,sku_15368.0,sku_9272.0,sku_325.0,sku_6357.0,sku_15338.0,sku_725.0,sku
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,False,0.0,False,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9288.0
0297fc1e,0.0,7.0,404.0,6.0,29.0,True,1.0,True,4.0,95.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2832.0
2d681dd8,0.0,1.0,13.0,1.0,1.0,True,0.0,False,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10084.0
cccea85e,0.0,1.0,739.0,1.0,15.0,True,0.0,False,7.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10407.0
4c8a8b93,0.0,2.0,177.0,9.0,14.0,True,0.0,False,8.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6873.0
29ebb414,0.0,1.0,8.0,11.0,4.0,True,0.0,False,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7153.0
3dc1950f,0.0,3.0,595.0,2.0,3.0,True,0.0,False,6.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3108.0
8ea4c165,0.0,3.0,97.0,17.0,13.0,True,0.0,False,5.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9288.0
d8cfe234,1.0,3.0,39.0,8.0,0.0,False,0.0,False,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,382.0


In [35]:
kaggle_pred = rf.predict_proba(df_events)
kaggle_pred
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [36]:
df_submit.to_csv('submit/submit.csv')


In [37]:
df_submit.label.mean()

0.27240724384026327

In [38]:
0.34460859613130107

0.34460859613130107